In [1]:
import sympy as sp

# --- Define symbols ---
theta, phi = sp.symbols('theta phi', real=True)
Mxx, Mxy, Mxz, Myy, Myz, Mzz = sp.symbols('M_xx M_xy M_xz M_yy M_yz M_zz', real=True)

# --- Define unit vector u(θ, φ) in spherical coordinates ---
ux = sp.sin(theta) * sp.cos(phi)
uy = sp.sin(theta) * sp.sin(phi)
uz = sp.cos(theta)
u = sp.Matrix([ux, uy, uz])

# --- Compute outer product u ⊗ u ---
u_outer_u = u * u.T  # This gives a 3x3 matrix

# --- Define symbolic second moment tensor <u ⊗ u> ---
M = sp.Matrix([
    [Mxx, Mxy, Mxz],
    [Mxy, Myy, Myz],
    [Mxz, Myz, Mzz]
])

# --- Compute contraction <u ⊗ u> : (u ⊗ u) = Tr(M · (u ⊗ u)) ---
contraction = sp.simplify(sp.trace(M * u_outer_u))

# --- Output ---
print("u(θ, φ) =")
sp.pprint(u, use_unicode=True)

print("\nu ⊗ u =")
sp.pprint(u_outer_u, use_unicode=True)

print("\n<u ⊗ u> : (u ⊗ u) =")
sp.pprint(contraction, use_unicode=True)


u(θ, φ) =
⎡sin(θ)⋅cos(φ)⎤
⎢             ⎥
⎢sin(φ)⋅sin(θ)⎥
⎢             ⎥
⎣   cos(θ)    ⎦

u ⊗ u =
⎡      2       2                  2                                ⎤
⎢   sin (θ)⋅cos (φ)     sin(φ)⋅sin (θ)⋅cos(φ)  sin(θ)⋅cos(φ)⋅cos(θ)⎥
⎢                                                                  ⎥
⎢          2                  2       2                            ⎥
⎢sin(φ)⋅sin (θ)⋅cos(φ)     sin (φ)⋅sin (θ)     sin(φ)⋅sin(θ)⋅cos(θ)⎥
⎢                                                                  ⎥
⎢                                                       2          ⎥
⎣sin(θ)⋅cos(φ)⋅cos(θ)   sin(φ)⋅sin(θ)⋅cos(θ)         cos (θ)       ⎦

<u ⊗ u> : (u ⊗ u) =
       2       2                       2             M_xz⋅(-sin(φ - 2⋅θ) + sin ↪
Mₓₓ⋅sin (θ)⋅cos (φ) + 2⋅M_xy⋅sin(φ)⋅sin (θ)⋅cos(φ) + ───────────────────────── ↪
                                                                      2        ↪

↪ (φ + 2⋅θ))           2       2      M_yz⋅(cos(φ - 2⋅θ) - cos(φ + 2⋅θ))       ↪
↪ ─

In [14]:
import sympy as sp


from sympy.simplify.fu import TR22, TR10, TR2, TR8, TR12, TR1

# Apply known trigonometric rewrite rules manually
def custom_full_simplify(expr):
    expr = sp.expand_trig(expr)               # Expand cos(a ± b)
    expr = sp.trigsimp(expr, method='fu')     # Full trigonometric simplification
    expr = TR22(expr)                         # cos(a) - cos(b) rules
    expr = TR10(expr)                         # sin(a)^2 + cos(a)^2 → 1
    expr = TR1(expr)                          # sin^2, cos^2 → identities
    expr = sp.simplify(expr)                  # General algebraic simplification
    return expr


# --- Define symbols ---
theta, phi, alpha, S = sp.symbols('theta phi alpha S', real=True)



# --- Define unit vector u(θ, φ) in spherical coordinates ---
ux = sp.sin(theta) * sp.cos(phi)
uy = sp.sin(theta) * sp.sin(phi)
uz = sp.cos(theta)
u = sp.Matrix([ux, uy, uz])

# --- Compute outer product u ⊗ u ---
u_outer_u = u * u.T  # This gives a 3x3 matrix

# --- Define symbolic second moment tensor <u ⊗ u> ---
M = sp.Matrix([
    (1-S)*sp.eye(3)/3 +
    S * sp.Matrix([
        [sp.cos(alpha)**2, 0, sp.sin(alpha)*sp.cos(alpha)],
        [0, 0, 0],
        [sp.sin(alpha)*sp.cos(alpha), 0, sp.sin(alpha)**2]
    ])

])

# --- Compute contraction <u ⊗ u> : (u ⊗ u) = Tr(M · (u ⊗ u)) ---
contraction = sp.trace(M * u_outer_u)

simplified_contraction = custom_full_simplify(contraction)


# # --- Output ---
# print("u(θ, φ) =")
# sp.pprint(u, use_unicode=True)

# print("\nu ⊗ u =")
# sp.pprint(u_outer_u, use_unicode=True)

# print("\n M =")
# sp.pprint(M, use_unicode=True)


print("\n<u ⊗ u> : (u ⊗ u) =")
sp.pprint(simplified_contraction, use_unicode=True, wrap_line=False)





<u ⊗ u> : (u ⊗ u) =
                                                                                                    2       2      ⎛       2           ⎞    2      ⎛       2           ⎞    2       2   
S⋅(cos(-2⋅α + φ + 2⋅θ) - cos(2⋅α - φ + 2⋅θ) + cos(2⋅α + φ - 2⋅θ) - cos(2⋅α + φ + 2⋅θ))   (1 - S)⋅sin (φ)⋅sin (θ)   ⎝3⋅S⋅sin (α) - S + 1⎠⋅cos (θ)   ⎝3⋅S⋅cos (α) - S + 1⎠⋅sin (θ)⋅cos (φ)
────────────────────────────────────────────────────────────────────────────────────── + ─────────────────────── + ───────────────────────────── + ─────────────────────────────────────
                                          8                                                         3                            3                                   3                  


In [10]:
from sympy import symbols, cos, sin, simplify

a, b = symbols('a b', real=True)
expr = cos(a - b) - cos(a + b)

simplified = simplify(expr)
print(simplified)  # Outputs: 2*sin(a)*sin(b)


2*sin(a)*sin(b)
